### 1. post-mdgw, oes completeness check

In [3]:
# SZ
import pandas as pd
import numpy as np

logSZ = pd.read_csv(r'F:\data\20200730\postMDGW_OES\mdLog_SZ_20200730_0913.csv')
print("data source type: ")
print(logSZ["source"].unique())
print(logSZ["time"].unique())

logSZ1 = logSZ[logSZ['source'] == 24]
logSZ = logSZ[logSZ['source'] == 4]
logSZ1["time"] = logSZ1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))

display(logSZ[(logSZ["StockID"] == 1) & (logSZ["time"] >= 93000000) & (logSZ["time"] <= 93500000)])
display(logSZ1[(logSZ1["StockID"] == 1) & (logSZ1["time"] >= 93000000) & (logSZ1["time"] <= 93500000)])

data1 = logSZ[(logSZ["cum_volume"] > 0) & (logSZ["time"] < 145700000) & (logSZ["source"] == 4)]
data2 = logSZ1[(logSZ1["cum_volume"] > 0) & (logSZ1["time"] < 145700000) & (logSZ1["source"] == 24)]
columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice"]
data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())
print(n1)
print(n2)
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])
    print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["sequenceNo_x"])])
    display(n2-n1)

test1 = test[~np.isnan(test["sequenceNo_x"])]
test1 = test1[~np.isnan(test1["sequenceNo_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

2229
2229
7973402
7973402
7973402
-----------------------------------------------
168.0
351.0
540.0
68.0
165.0
3193.0
0.7412316599614569
0.2576609081042195


In [6]:
# Order + trade
import pandas as pd
import random
import numpy as np
import pickle

TradeLog = pd.read_csv(r'F:\data\20200730\postMDGW_OES\mdTradeLog_20200730_0913.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "mdSource", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
TradeLog1 = TradeLog[TradeLog['mdSource'] == 24]
TradeLog = TradeLog[TradeLog['mdSource'] == 4]
OrderLog = pd.read_csv(r'F:\data\20200730\postMDGW_OES\mdOrderLog_20200730_0913.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "mdSource", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
OrderLog1 = OrderLog[OrderLog['mdSource'] == 24]
OrderLog = OrderLog[OrderLog['mdSource'] == 4]

stocks = OrderLog1["SecurityID"].unique()
l1 = stocks[:1000]
l2 = stocks[1000:]
OrderLogSZ = OrderLog[OrderLog["SecurityID"].isin(l1)]
OrderLogSZ1 = OrderLog1[OrderLog1["SecurityID"].isin(l1)]
print(len(OrderLogSZ["SecurityID"].unique()))
print(len(OrderLogSZ1["SecurityID"].unique()))
OrderLogSZZ = OrderLog[OrderLog["SecurityID"].isin(l2)]
OrderLogSZZ1 = OrderLog1[OrderLog1["SecurityID"].isin(l2)]
print(len(OrderLogSZZ["SecurityID"].unique()))
print(len(OrderLogSZZ1["SecurityID"].unique()))
del OrderLog
del OrderLog1
TradeLogSZ = TradeLog[TradeLog["SecurityID"].isin(l1)]
TradeLogSZ1 = TradeLog1[TradeLog1["SecurityID"].isin(l1)]
print(len(TradeLogSZ["SecurityID"].unique()))
print(len(TradeLogSZ1["SecurityID"].unique()))
TradeLogSZZ = TradeLog[TradeLog["SecurityID"].isin(l2)]
TradeLogSZZ1 = TradeLog1[TradeLog1["SecurityID"].isin(l2)]
print(len(TradeLogSZZ["SecurityID"].unique()))
print(len(TradeLogSZZ1["SecurityID"].unique()))
del TradeLog
del TradeLog1

OrderLogSZ.to_pickle(r'F:\\data\\20200730\\baseline\\order1.pkl')
OrderLogSZZ.to_pickle(r'F:\\data\\20200730\\baseline\\order2.pkl')
TradeLogSZ.to_pickle(r'F:\\data\\20200730\\baseline\\trade1.pkl')
TradeLogSZZ.to_pickle(r'F:\\data\\20200730\\baseline\\trade2.pkl')
OrderLogSZ1.to_pickle(r'F:\\data\\20200730\\test\\order1.pkl')
OrderLogSZZ1.to_pickle(r'F:\\data\\20200730\\test\\order2.pkl')
TradeLogSZ1.to_pickle(r'F:\\data\\20200730\\test\\trade1.pkl')
TradeLogSZZ1.to_pickle(r'F:\\data\\20200730\\test\\trade2.pkl')

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1000
1000
1229
1229
1000
1000
1229
1229


In [1]:
import pandas as pd
import random
import numpy as np
import pickle

F1 = open("F:\\data\\20200730\\baseline\\trade1.pkl", 'rb')
TradeLogSZ = pickle.load(F1)
F2 = open("F:\\data\\20200730\\test\\trade1.pkl", 'rb')
TradeLogSZ1 = pickle.load(F2)

sl = list(set(TradeLogSZ["SecurityID"].unique()) & set(TradeLogSZ['SecurityID'].unique()))
TradeLogSZ = TradeLogSZ[TradeLogSZ["SecurityID"].isin(sl)]
TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1["SecurityID"].isin(sl)]
print(len(TradeLogSZ["SecurityID"].unique()))
print(len(TradeLogSZ1["SecurityID"].unique()))

F1 = open("F:\\data\\20200730\\baseline\\trade2.pkl", 'rb')
t = pickle.load(F1)
F2 = open("F:\\data\\20200730\\test\\trade2.pkl", 'rb')
t1 = pickle.load(F2)
sl = list(set(t["SecurityID"].unique()) & set(t1['SecurityID'].unique()))
t = t[t["SecurityID"].isin(sl)]
t1 = t1[t1["SecurityID"].isin(sl)]
print(len(t["SecurityID"].unique()))
print(len(t1["SecurityID"].unique()))

TradeLogSZ = pd.concat([TradeLogSZ, t])
TradeLogSZ1 = pd.concat([TradeLogSZ1, t1])
del t
del t1

TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

re = re[~re["sequenceNo_x"].isnull()]
re = re[~re["sequenceNo_y"].isnull()]
diff = re["clockAtArrival_y"] - re["clockAtArrival_x"]
re["diff"] = re["clockAtArrival_y"] - re["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))
print(sum(diff == 0)/len(diff))

1000
1000
1229
1229
46297588
46297588
46297588
-----------------------------------------------
47.0
77.0
123.0
48.0
123.0
259.0
0.6307602676839235
0.36685321490182166
0.0023865174142549284


In [1]:
import pandas as pd
import random
import numpy as np
import pickle

F1 = open("F:\\data\\20200730\\baseline\\order1.pkl", 'rb')
OrderLogSZ = pickle.load(F1)
F2 = open("F:\\data\\20200730\\test\\order1.pkl", 'rb')
OrderLogSZ1 = pickle.load(F2)

OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
OrderLogSZ1["OrderType"] = OrderLogSZ1["OrderType"].apply(lambda x: str(x))
sl = list(set(OrderLogSZ["SecurityID"].unique()) & set(OrderLogSZ1['SecurityID'].unique()))
OrderLogSZ = OrderLogSZ[OrderLogSZ["SecurityID"].isin(sl)]
OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["SecurityID"].isin(sl)]
print(len(OrderLogSZ["SecurityID"].unique()))
print(len(OrderLogSZ1["SecurityID"].unique()))

F1 = open("F:\\data\\20200730\\baseline\\order2.pkl", 'rb')
o = pickle.load(F1)
F2 = open("F:\\data\\20200730\\test\\order2.pkl", 'rb')
o1 = pickle.load(F2)
o["OrderType"] = o["OrderType"].apply(lambda x: str(x))
o1["OrderType"] = o1["OrderType"].apply(lambda x: str(x))
sl = list(set(o["SecurityID"].unique()) & set(o1['SecurityID'].unique()))
o = o[o["SecurityID"].isin(sl)]
o1 = o1[o1["SecurityID"].isin(sl)]
print(len(o["SecurityID"].unique()))
print(len(o1["SecurityID"].unique()))

OrderLogSZ = pd.concat([OrderLogSZ, o])
OrderLogSZ1 = pd.concat([OrderLogSZ1, o1])
del o
del o1

columns = ["TransactTime", "ApplSeqNum","Side",'OrderType', 'Price', 'OrderQty', "SecurityID"]
ree = pd.merge(OrderLogSZ, OrderLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = ree["sequenceNo_x"].count()
n2 = ree["sequenceNo_y"].count()
len1 = len(ree)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(ree[np.isnan(ree["sequenceNo_y"])])
    print(len(ree[np.isnan(ree["sequenceNo_y"])]))
    print(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
    print(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("test is complete, baseline is not complete:")
    display(ree[np.isnan(ree["sequenceNo_x"])])
    print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
    print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)
    
ree = ree[~ree["sequenceNo_x"].isnull()]
ree = ree[~ree["sequenceNo_y"].isnull()]
ree["diff"] = ree["clockAtArrival_y"] - ree["clockAtArrival_x"]
diff = ree["clockAtArrival_y"] - ree["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

1000
1000
1229
1229
51781800
51781801
51781801
-----------------------------------------------


'test is complete, baseline is not complete:'

,clockAtArrival_x,sequenceNo_x,exchId_x,mdSource_x,TransactTime,ApplSeqNum,SecurityID,Side,OrderType,Price,OrderQty,clockAtArrival_y,sequenceNo_y,exchId_y,mdSource_y
51781800,NaN,NaN,NaN,NaN,145959960,24931711,166,1,2,57700,200,1596092402869124,250799443,2,24


[145959960]
1
[166]


1

47.0
72.0
116.0
50.0
125.0
266.0
0.6365825637579227
0.36115389963268946


### 2. pre-mdgw, post-mdgw completeness check

In [1]:
# SZ
import pandas as pd
import numpy as np

logSZ = pd.read_csv(r'F:\data\20200730\postMDGW_OES\mdLog_SZ_20200730_0913.csv')
logSZ = logSZ[logSZ['source'] == 24]
logSZ1 = pd.read_csv(r'F:\data\20200730\preMDGW\PCAP\mdLog_SZ_20200730_1814.csv')

logSZ1["time"] = logSZ1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))

display(logSZ[(logSZ["StockID"] == 1) & (logSZ["time"] >= 93000000) & (logSZ["time"] <= 93500000)])
display(logSZ1[(logSZ1["StockID"] == 1) & (logSZ1["time"] >= 93000000) & (logSZ1["time"] <= 93500000)])

data1 = logSZ[(logSZ["cum_volume"] > 0) & (logSZ["time"] < 145700000) & (logSZ["source"] == 24)]
data2 = logSZ1[(logSZ1["cum_volume"] > 0) & (logSZ1["time"] < 145700000) & (logSZ1["source"] == 24)]
columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice"]
data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())
print(n1)
print(n2)
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])
    print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["sequenceNo_x"])])
    display(n2-n1)

,ms,clock,threadId,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,...,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
287517,09:30:03.345773,1596072603346235,435259,1596072603346209,3799355,24,1,SZ,93000000,942800,...,13.52,13.53,13.54,115912,403900,162200,303088,465100,13.5,524
291744,09:30:06.491773,1596072606492053,435259,1596072606492017,4034450,24,1,SZ,93003000,1499900,...,13.51,13.52,13.53,184000,233312,437100,108400,362288,13.5,810
298450,09:30:09.471773,1596072609472126,435259,1596072609472108,4247486,24,1,SZ,93006000,1915100,...,13.52,13.53,13.54,331112,442800,111400,337688,156100,13.5,939
305121,09:30:12.437773,1596072612438100,435259,1596072612438067,4464903,24,1,SZ,93009000,1969600,...,13.51,13.52,13.53,3300,373712,442600,113600,337288,13.5,999
311803,09:30:15.425774,1596072615426628,435259,1596072615426592,4694107,24,1,SZ,93012000,2011600,...,13.50,13.51,13.52,1400,16700,403412,431800,115300,13.5,1041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903065,09:34:51.412786,1596072891413883,435259,1596072891413838,21743954,24,1,SZ,93448000,7021500,...,13.47,13.48,13.49,62600,152600,40200,151400,91410,13.5,3720
909377,09:34:54.425786,1596072894426852,435259,1596072894426830,21865455,24,1,SZ,93451000,7088300,...,13.47,13.48,13.49,65600,150700,40200,151400,90310,13.5,3752
915592,09:34:57.399786,1596072897400252,435259,1596072897400224,21982899,24,1,SZ,93454000,7197200,...,13.47,13.48,13.49,108300,147900,42400,153800,94110,13.5,3811
921867,09:35:00.411786,1596072900412859,435259,1596072900412843,22099487,24,1,SZ,93457000,7229200,...,13.47,13.48,13.49,122700,146000,42400,153800,94110,13.5,3835


,ms,clock,threadId,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,...,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
416062,18:14:40.656716,1596104080657493,6017,1596104080657464,2013717,24,1,SZ,93000000,942800,...,13.52,13.53,13.54,115912,403900,162200,303088,465100,13.5,524
417470,18:14:40.965715,1596104080966618,6017,1596104080966583,2112534,24,1,SZ,93003000,1499900,...,13.51,13.52,13.53,184000,233312,437100,108400,362288,13.5,810
419697,18:14:41.255716,1596104081256216,6017,1596104081256176,2208795,24,1,SZ,93006000,1915100,...,13.52,13.53,13.54,331112,442800,111400,337688,156100,13.5,939
421891,18:14:41.542733,1596104081543093,6017,1596104081543028,2304419,24,1,SZ,93009000,1969600,...,13.51,13.52,13.53,3300,373712,442600,113600,337288,13.5,999
424100,18:14:41.828874,1596104081829737,6017,1596104081829702,2400944,24,1,SZ,93012000,2011600,...,13.50,13.51,13.52,1400,16700,403412,431800,115300,13.5,1041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620795,18:15:05.179020,1596104105179443,6017,1596104105179345,9871184,24,1,SZ,93448000,7021500,...,13.47,13.48,13.49,62600,152600,40200,151400,91410,13.5,3720
622910,18:15:05.357017,1596104105357418,6017,1596104105357393,9923148,24,1,SZ,93451000,7088300,...,13.47,13.48,13.49,65600,150700,40200,151400,90310,13.5,3752
624981,18:15:05.530565,1596104105531147,6017,1596104105531112,9973392,24,1,SZ,93454000,7197200,...,13.47,13.48,13.49,108300,147900,42400,153800,94110,13.5,3811
627076,18:15:05.707071,1596104105707343,6017,1596104105707206,10023868,24,1,SZ,93457000,7229200,...,13.47,13.48,13.49,122700,146000,42400,153800,94110,13.5,3835


2229
2210
7897752
7897752
7897752
-----------------------------------------------


In [1]:
# Order + trade
import pandas as pd
import random
import numpy as np
import pickle

TradeLog = pd.read_csv(r'F:\data\20200730\postMDGW_OES\mdTradeLog_20200730_0913.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "mdSource", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
TradeLog = TradeLog[TradeLog['mdSource'] == 24]
OrderLog = pd.read_csv(r'F:\data\20200730\postMDGW_OES\mdOrderLog_20200730_0913.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "mdSource", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
OrderLog = OrderLog[OrderLog['mdSource'] == 24]
TradeLog1 = pd.read_csv(r'F:\data\20200730\preMDGW\PCAP\mdTradeLog_20200730_1814.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "mdSource", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
OrderLog1 = pd.read_csv(r'F:\data\20200730\preMDGW\PCAP\mdOrderLog_20200730_1814.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "mdSource", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]

stocks = OrderLog1["SecurityID"].unique()
l1 = stocks[:1000]
l2 = stocks[1000:]
OrderLogSZ = OrderLog[OrderLog["SecurityID"].isin(l1)]
OrderLogSZ1 = OrderLog1[OrderLog1["SecurityID"].isin(l1)]
print(len(OrderLogSZ["SecurityID"].unique()))
print(len(OrderLogSZ1["SecurityID"].unique()))
OrderLogSZZ = OrderLog[OrderLog["SecurityID"].isin(l2)]
OrderLogSZZ1 = OrderLog1[OrderLog1["SecurityID"].isin(l2)]
print(len(OrderLogSZZ["SecurityID"].unique()))
print(len(OrderLogSZZ1["SecurityID"].unique()))
del OrderLog
del OrderLog1
TradeLogSZ = TradeLog[TradeLog["SecurityID"].isin(l1)]
TradeLogSZ1 = TradeLog1[TradeLog1["SecurityID"].isin(l1)]
print(len(TradeLogSZ["SecurityID"].unique()))
print(len(TradeLogSZ1["SecurityID"].unique()))
TradeLogSZZ = TradeLog[TradeLog["SecurityID"].isin(l2)]
TradeLogSZZ1 = TradeLog1[TradeLog1["SecurityID"].isin(l2)]
print(len(TradeLogSZZ["SecurityID"].unique()))
print(len(TradeLogSZZ1["SecurityID"].unique()))
del TradeLog
del TradeLog1

OrderLogSZ.to_pickle(r'F:\\data\\20200730\\baseline\\order1.pkl')
OrderLogSZZ.to_pickle(r'F:\\data\\20200730\\baseline\\order2.pkl')
TradeLogSZ.to_pickle(r'F:\\data\\20200730\\baseline\\trade1.pkl')
TradeLogSZZ.to_pickle(r'F:\\data\\20200730\\baseline\\trade2.pkl')
OrderLogSZ1.to_pickle(r'F:\\data\\20200730\\test\\order1.pkl')
OrderLogSZZ1.to_pickle(r'F:\\data\\20200730\\test\\order2.pkl')
TradeLogSZ1.to_pickle(r'F:\\data\\20200730\\test\\trade1.pkl')
TradeLogSZZ1.to_pickle(r'F:\\data\\20200730\\test\\trade2.pkl')

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1000
1000
1210
1210
1000
1000
1210
1210


In [1]:
import pandas as pd
import random
import numpy as np
import pickle

F1 = open("F:\\data\\20200730\\baseline\\trade1.pkl", 'rb')
TradeLogSZ = pickle.load(F1)
F2 = open("F:\\data\\20200730\\test\\trade1.pkl", 'rb')
TradeLogSZ1 = pickle.load(F2)

sl = list(set(TradeLogSZ["SecurityID"].unique()) & set(TradeLogSZ['SecurityID'].unique()))
TradeLogSZ = TradeLogSZ[TradeLogSZ["SecurityID"].isin(sl)]
TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1["SecurityID"].isin(sl)]
print(len(TradeLogSZ["SecurityID"].unique()))
print(len(TradeLogSZ1["SecurityID"].unique()))

F1 = open("F:\\data\\20200730\\baseline\\trade2.pkl", 'rb')
t = pickle.load(F1)
F2 = open("F:\\data\\20200730\\test\\trade2.pkl", 'rb')
t1 = pickle.load(F2)
sl = list(set(t["SecurityID"].unique()) & set(t1['SecurityID'].unique()))
t = t[t["SecurityID"].isin(sl)]
t1 = t1[t1["SecurityID"].isin(sl)]
print(len(t["SecurityID"].unique()))
print(len(t1["SecurityID"].unique()))

TradeLogSZ = pd.concat([TradeLogSZ, t])
TradeLogSZ1 = pd.concat([TradeLogSZ1, t1])
del t
del t1

TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

1000
1000
1210
1210
45773657
45773657
45773657
-----------------------------------------------


In [1]:
import pandas as pd
import random
import numpy as np
import pickle

F1 = open("F:\\data\\20200730\\baseline\\order1.pkl", 'rb')
OrderLogSZ = pickle.load(F1)
F2 = open("F:\\data\\20200730\\test\\order1.pkl", 'rb')
OrderLogSZ1 = pickle.load(F2)

OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
OrderLogSZ1["OrderType"] = OrderLogSZ1["OrderType"].apply(lambda x: str(x))
sl = list(set(OrderLogSZ["SecurityID"].unique()) & set(OrderLogSZ1['SecurityID'].unique()))
OrderLogSZ = OrderLogSZ[OrderLogSZ["SecurityID"].isin(sl)]
OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["SecurityID"].isin(sl)]
print(len(OrderLogSZ["SecurityID"].unique()))
print(len(OrderLogSZ1["SecurityID"].unique()))

F1 = open("F:\\data\\20200730\\baseline\\order2.pkl", 'rb')
o = pickle.load(F1)
F2 = open("F:\\data\\20200730\\test\\order2.pkl", 'rb')
o1 = pickle.load(F2)
o["OrderType"] = o["OrderType"].apply(lambda x: str(x))
o1["OrderType"] = o1["OrderType"].apply(lambda x: str(x))
sl = list(set(o["SecurityID"].unique()) & set(o1['SecurityID'].unique()))
o = o[o["SecurityID"].isin(sl)]
o1 = o1[o1["SecurityID"].isin(sl)]
print(len(o["SecurityID"].unique()))
print(len(o1["SecurityID"].unique()))

OrderLogSZ = pd.concat([OrderLogSZ, o])
OrderLogSZ1 = pd.concat([OrderLogSZ1, o1])
del o
del o1

columns = ["TransactTime", "ApplSeqNum","Side",'OrderType', 'Price', 'OrderQty', "SecurityID"]
ree = pd.merge(OrderLogSZ, OrderLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = ree["sequenceNo_x"].count()
n2 = ree["sequenceNo_y"].count()
len1 = len(ree)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(ree[np.isnan(ree["sequenceNo_y"])])
    print(len(ree[np.isnan(ree["sequenceNo_y"])]))
    print(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
    print(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("test is complete, baseline is not complete:")
    display(ree[np.isnan(ree["sequenceNo_x"])])
    print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
    print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

1000
1000
1210
1210
51206776
51206814
51206814
-----------------------------------------------


'test is complete, baseline is not complete:'

,clockAtArrival_x,sequenceNo_x,exchId_x,mdSource_x,TransactTime,ApplSeqNum,SecurityID,Side,OrderType,Price,OrderQty,clockAtArrival_y,sequenceNo_y,exchId_y,mdSource_y
51206776,NaN,NaN,NaN,NaN,145959960,24236735,300759,1,2,1081900,100,1596104511769226,105683016,2,24
51206777,NaN,NaN,NaN,NaN,145959960,23810403,2457,1,2,104400,500,1596104511769229,105683017,2,24
51206778,NaN,NaN,NaN,NaN,145959970,23810405,2208,1,2,123200,900,1596104511769246,105683021,2,24
51206779,NaN,NaN,NaN,NaN,145959970,23810406,2465,1,2,146000,1000,1596104511769261,105683023,2,24
51206780,NaN,NaN,NaN,NaN,145959970,23810408,2567,1,2,122500,5000,1596104511769277,105683025,2,24
51206781,NaN,NaN,NaN,NaN,145959970,24236736,300664,2,2,102600,200,1596104511769282,105683027,2,24
51206782,NaN,NaN,NaN,NaN,145959970,24931713,300059,2,2,261000,1000,1596104511769289,105683029,2,24
51206783,NaN,NaN,NaN,NaN,145959970,24236737,961,2,2,95500,3100,1596104511769292,105683030,2,24
51206784,NaN,NaN,NaN,NaN,145959980,23810410,625,1,2,110200,100,1596104511769296,105683031,2,24
51206785,NaN,NaN,NaN,NaN,145959980,24931714,540,1,2,35400,1500,1596104511769300,105683032,2,24


[145959960 145959970 145959980 145959990]
38
[300759   2457   2208   2465   2567 300664 300059    961    625    540
   2626   2299   2286 300775   2069   2602 300045 300797    915 300005
    988 300726   2235 300589   2885   2923   2506 300370   2428 300403
   2685   2035    829 300105   2930   2643   2119 300346]


38

### 3. pre-mdgw, post-mdgw speed check

In [10]:
# SZ
import pandas as pd
import numpy as np

logSZ = pd.read_csv(r'F:\data\20200730\postMDGW_OES\mdLog_SZ_20200730_0913.csv')
logSZ = logSZ[logSZ['source'] == 24]
logSZ1 = pd.read_csv(r'F:\data\20200730\preMDGW\EXE\Logs_full\mdLog_SZ_20200730_0929.csv')

logSZ1["time"] = logSZ1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))

display(logSZ[(logSZ["StockID"] == 1) & (logSZ["time"] >= 93000000) & (logSZ["time"] <= 93500000)])
display(logSZ1[(logSZ1["StockID"] == 1) & (logSZ1["time"] >= 93000000) & (logSZ1["time"] <= 93500000)])

data1 = logSZ[(logSZ["cum_volume"] > 0) & (logSZ['time'] >= 93000000) & (logSZ["time"] < 145700000) & (logSZ["source"] == 24)]
data2 = logSZ1[(logSZ1["cum_volume"] > 0) & (logSZ1['time'] >= 93000000) & (logSZ1["time"] < 145700000) & (logSZ1["source"] == 24)]
columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice"]
data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())
print(n1)
print(n2)
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])
    print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
    display(test[np.isnan(test["sequenceNo_y"])]['time_x'].describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))
    display(data2_1.groupby('StockID')['time'].min().max())
    display(test[np.isnan(test["sequenceNo_y"])]['StockID'].unique())
    display(test[np.isnan(test["sequenceNo_y"])]['time_x'].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["sequenceNo_x"])])
    display(n2-n1)

test1 = test[~np.isnan(test["sequenceNo_x"])]
test1 = test1[~np.isnan(test1["sequenceNo_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

2229
2210
7895627
7895627
7895627
-----------------------------------------------
675.0
3639.0
10814.0
781.0
923.0
2029.0
0.9999887279376293
1.1272062370727493e-05


In [1]:
# Order + trade
import pandas as pd
import random
import numpy as np
import pickle

TradeLog = pd.read_csv(r'F:\data\20200730\postMDGW_OES\mdTradeLog_20200730_0913.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "mdSource", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
TradeLog = TradeLog[TradeLog['mdSource'] == 24]
OrderLog = pd.read_csv(r'F:\data\20200730\postMDGW_OES\mdOrderLog_20200730_0913.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "mdSource", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
OrderLog = OrderLog[OrderLog['mdSource'] == 24]
TradeLog1 = pd.read_csv(r'F:\data\20200730\preMDGW\EXE\Logs_full\mdTradeLog_20200730_0929.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "mdSource", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
OrderLog1 = pd.read_csv(r'F:\data\20200730\preMDGW\EXE\Logs_full\mdOrderLog_20200730_0929.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "mdSource", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]

stocks = OrderLog1["SecurityID"].unique()
l1 = stocks[:1000]
l2 = stocks[1000:]
OrderLogSZ = OrderLog[OrderLog["SecurityID"].isin(l1)]
OrderLogSZ1 = OrderLog1[OrderLog1["SecurityID"].isin(l1)]
print(len(OrderLogSZ["SecurityID"].unique()))
print(len(OrderLogSZ1["SecurityID"].unique()))
OrderLogSZZ = OrderLog[OrderLog["SecurityID"].isin(l2)]
OrderLogSZZ1 = OrderLog1[OrderLog1["SecurityID"].isin(l2)]
print(len(OrderLogSZZ["SecurityID"].unique()))
print(len(OrderLogSZZ1["SecurityID"].unique()))
del OrderLog
del OrderLog1
TradeLogSZ = TradeLog[TradeLog["SecurityID"].isin(l1)]
TradeLogSZ1 = TradeLog1[TradeLog1["SecurityID"].isin(l1)]
print(len(TradeLogSZ["SecurityID"].unique()))
print(len(TradeLogSZ1["SecurityID"].unique()))
TradeLogSZZ = TradeLog[TradeLog["SecurityID"].isin(l2)]
TradeLogSZZ1 = TradeLog1[TradeLog1["SecurityID"].isin(l2)]
print(len(TradeLogSZZ["SecurityID"].unique()))
print(len(TradeLogSZZ1["SecurityID"].unique()))
del TradeLog
del TradeLog1

OrderLogSZ.to_pickle(r'F:\\data\\20200730\\baseline\\order1.pkl')
OrderLogSZZ.to_pickle(r'F:\\data\\20200730\\baseline\\order2.pkl')
TradeLogSZ.to_pickle(r'F:\\data\\20200730\\baseline\\trade1.pkl')
TradeLogSZZ.to_pickle(r'F:\\data\\20200730\\baseline\\trade2.pkl')
OrderLogSZ1.to_pickle(r'F:\\data\\20200730\\test\\order1.pkl')
OrderLogSZZ1.to_pickle(r'F:\\data\\20200730\\test\\order2.pkl')
TradeLogSZ1.to_pickle(r'F:\\data\\20200730\\test\\trade1.pkl')
TradeLogSZZ1.to_pickle(r'F:\\data\\20200730\\test\\trade2.pkl')

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1000
1000
1210
1210
1000
1000
1210
1210


In [6]:
import pandas as pd
import random
import numpy as np
import pickle

F1 = open("F:\\data\\20200730\\baseline\\trade1.pkl", 'rb')
TradeLogSZ = pickle.load(F1)
F2 = open("F:\\data\\20200730\\test\\trade1.pkl", 'rb')
TradeLogSZ1 = pickle.load(F2)

sl = list(set(TradeLogSZ["SecurityID"].unique()) & set(TradeLogSZ['SecurityID'].unique()))
TradeLogSZ = TradeLogSZ[TradeLogSZ["SecurityID"].isin(sl)]
TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1["SecurityID"].isin(sl)]
print(len(TradeLogSZ["SecurityID"].unique()))
print(len(TradeLogSZ1["SecurityID"].unique()))

F1 = open("F:\\data\\20200730\\baseline\\trade2.pkl", 'rb')
t = pickle.load(F1)
F2 = open("F:\\data\\20200730\\test\\trade2.pkl", 'rb')
t1 = pickle.load(F2)
sl = list(set(t["SecurityID"].unique()) & set(t1['SecurityID'].unique()))
t = t[t["SecurityID"].isin(sl)]
t1 = t1[t1["SecurityID"].isin(sl)]
print(len(t["SecurityID"].unique()))
print(len(t1["SecurityID"].unique()))

TradeLogSZ = pd.concat([TradeLogSZ, t])
TradeLogSZ1 = pd.concat([TradeLogSZ1, t1])
del t
del t1

TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]

TradeLogSZ = TradeLogSZ[TradeLogSZ['TransactTime'] >= 93000000]
TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1['TransactTime'] >= 93000000]

re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

re = re[~re["sequenceNo_x"].isnull()]
re = re[~re["sequenceNo_y"].isnull()]
diff = re["clockAtArrival_y"] - re["clockAtArrival_x"]
re["diff"] = re["clockAtArrival_y"] - re["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))
print(sum(diff == 0)/len(diff))

45533443
45533419
45533443
-----------------------------------------------


'test is not complete:'

,clockAtArrival_x,sequenceNo_x,exchId_x,mdSource_x,TransactTime,ApplSeqNum,SecurityID,ExecType,TradeBSFlag,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum,clockAtArrival_y,sequenceNo_y,exchId_y,mdSource_y
30900152,1596092409473262,251435952,2,24,150000000,25001759,300836,F,N,635400,1900,1207260000,24929326,24922275,NaN,NaN,NaN,NaN
30900153,1596092409473262,251435953,2,24,150000000,25001760,300836,F,N,635400,600,381240000,24836134,24922275,NaN,NaN,NaN,NaN
30900154,1596092409473308,251435954,2,24,150000000,25001761,300836,F,N,635400,200,127080000,24843925,24922275,NaN,NaN,NaN,NaN
30900155,1596092409473310,251435955,2,24,150000000,25001762,300836,F,N,635400,100,63540000,24855802,24922275,NaN,NaN,NaN,NaN
30900156,1596092409473311,251435956,2,24,150000000,25001763,300836,F,N,635400,200,127080000,24860194,24922275,NaN,NaN,NaN,NaN
30900157,1596092409473311,251435957,2,24,150000000,25001764,300836,F,N,635400,100,63540000,24861108,24922275,NaN,NaN,NaN,NaN
30900158,1596092409473312,251435958,2,24,150000000,25001765,300836,F,N,635400,200,127080000,24866997,24922275,NaN,NaN,NaN,NaN
30900159,1596092409473388,251435959,2,24,150000000,25001766,300836,F,N,635400,800,508320000,24870040,24922275,NaN,NaN,NaN,NaN
30900160,1596092409473390,251435960,2,24,150000000,25001767,300836,F,N,635400,200,127080000,24874955,24922275,NaN,NaN,NaN,NaN
30900161,1596092409473391,251435961,2,24,150000000,25001768,300836,F,N,635400,200,127080000,24880082,24922275,NaN,NaN,NaN,NaN


24
[150000000]
1
[300836]
690.0
802.0
1681.0
399.25
907.0
1318.5
0.9999886676640732
1.1288412144056214e-05
4.3923782661697334e-08


In [3]:
import pandas as pd
import random
import numpy as np
import pickle

# F1 = open("F:\\data\\20200730\\baseline\\order1.pkl", 'rb')
# OrderLogSZ = pickle.load(F1)
# F2 = open("F:\\data\\20200730\\test\\order1.pkl", 'rb')
# OrderLogSZ1 = pickle.load(F2)

# OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
# OrderLogSZ1["OrderType"] = OrderLogSZ1["OrderType"].apply(lambda x: str(x))
# sl = list(set(OrderLogSZ["SecurityID"].unique()) & set(OrderLogSZ1['SecurityID'].unique()))
# OrderLogSZ = OrderLogSZ[OrderLogSZ["SecurityID"].isin(sl)]
# OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["SecurityID"].isin(sl)]
# print(len(OrderLogSZ["SecurityID"].unique()))
# print(len(OrderLogSZ1["SecurityID"].unique()))

# F1 = open("F:\\data\\20200730\\baseline\\order2.pkl", 'rb')
# o = pickle.load(F1)
# F2 = open("F:\\data\\20200730\\test\\order2.pkl", 'rb')
# o1 = pickle.load(F2)
# o["OrderType"] = o["OrderType"].apply(lambda x: str(x))
# o1["OrderType"] = o1["OrderType"].apply(lambda x: str(x))
# sl = list(set(o["SecurityID"].unique()) & set(o1['SecurityID'].unique()))
# o = o[o["SecurityID"].isin(sl)]
# o1 = o1[o1["SecurityID"].isin(sl)]
# print(len(o["SecurityID"].unique()))
# print(len(o1["SecurityID"].unique()))

# OrderLogSZ = pd.concat([OrderLogSZ, o])
# OrderLogSZ1 = pd.concat([OrderLogSZ1, o1])
# del o
# del o1

columns = ["TransactTime", "ApplSeqNum","Side",'OrderType', 'Price', 'OrderQty', "SecurityID"]
OrderLogSZ = OrderLogSZ[OrderLogSZ['TransactTime'] >= 93000000]
OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1['TransactTime'] >= 93000000]
ree = pd.merge(OrderLogSZ, OrderLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = ree["sequenceNo_x"].count()
n2 = ree["sequenceNo_y"].count()
len1 = len(ree)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(ree[np.isnan(ree["sequenceNo_y"])])
    print(len(ree[np.isnan(ree["sequenceNo_y"])]))
    print(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
    print(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("test is complete, baseline is not complete:")
    display(ree[np.isnan(ree["sequenceNo_x"])])
    print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
    print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)
    
ree = ree[~ree["sequenceNo_x"].isnull()]
ree = ree[~ree["sequenceNo_y"].isnull()]
ree["diff"] = ree["clockAtArrival_y"] - ree["clockAtArrival_x"]
diff = ree["clockAtArrival_y"] - ree["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

49861686
49861709
49861709
-----------------------------------------------


'test is complete, baseline is not complete:'

,clockAtArrival_x,sequenceNo_x,exchId_x,mdSource_x,TransactTime,ApplSeqNum,SecurityID,Side,OrderType,Price,OrderQty,clockAtArrival_y,sequenceNo_y,exchId_y,mdSource_y
49861686,NaN,NaN,NaN,NaN,145959970,24931712,2885,1,2,171800,1200,1596092402870869,103681656,2,24
49861687,NaN,NaN,NaN,NaN,145959970,23810406,2465,1,2,146000,1000,1596092402871472,103681657,2,24
49861688,NaN,NaN,NaN,NaN,145959970,23810407,2923,1,2,231600,200,1596092402876179,103681658,2,24
49861689,NaN,NaN,NaN,NaN,145959970,23810408,2567,1,2,122500,5000,1596092402876908,103681659,2,24
49861690,NaN,NaN,NaN,NaN,145959970,24872064,2506,1,2,30900,600,1596092402877115,103681660,2,24
49861691,NaN,NaN,NaN,NaN,145959970,24236736,300664,2,2,102600,200,1596092402878335,103681661,2,24
49861692,NaN,NaN,NaN,NaN,145959970,23810409,300370,1,2,25600,3000,1596092402878751,103681662,2,24
49861693,NaN,NaN,NaN,NaN,145959970,24931713,300059,2,2,261000,1000,1596092402880325,103681663,2,24
49861694,NaN,NaN,NaN,NaN,145959970,24236737,961,2,2,95500,3100,1596092402880871,103681664,2,24
49861695,NaN,NaN,NaN,NaN,145959980,23810410,625,1,2,110200,100,1596092402881600,103681665,2,24


[145959960 145959970 145959980]
23
[  2885   2465   2923   2567   2506 300664 300370 300059    961    625
    540   2626   2428   2286 300775   2069   2602 300045 300759   2457
 300589   2208   2299]


23

664.0
772.0
1704.0
359.75
769.5
1313.0
0.9999879867680367
1.1993176484244836e-05


### 4. pre-mdgw, oes speed check

In [6]:
# SZ
import pandas as pd
import numpy as np

logSZ = pd.read_csv(r'F:\data\20200730\postMDGW_OES\mdLog_SZ_20200730_0913.csv')
logSZ = logSZ[logSZ['source'] == 4]
logSZ1 = pd.read_csv(r'F:\data\20200730\preMDGW\EXE\Logs_full\mdLog_SZ_20200730_0929.csv')

logSZ1["time"] = logSZ1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))

display(logSZ[(logSZ["StockID"] == 1) & (logSZ["time"] >= 93000000) & (logSZ["time"] <= 93500000)])
display(logSZ1[(logSZ1["StockID"] == 1) & (logSZ1["time"] >= 93000000) & (logSZ1["time"] <= 93500000)])

data1 = logSZ[(logSZ["cum_volume"] > 0) & (logSZ["time"] < 145700000) & (logSZ['time'] >= 93000000) & (logSZ["source"] == 4)]
data2 = logSZ1[(logSZ1["cum_volume"] > 0) & (logSZ1["time"] < 145700000) & (logSZ1['time'] >= 93000000) & (logSZ1["source"] == 24)]
columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice"]
data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())
print(n1)
print(n2)
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])
    print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
    display(test[np.isnan(test["sequenceNo_y"])]['time_x'].describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).astype(int))
    display(data2_1.groupby('StockID')['time'].min().max())
    display(test[np.isnan(test["sequenceNo_y"])]['StockID'].unique())
    display(test[np.isnan(test["sequenceNo_y"])]['time_x'].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["sequenceNo_x"])])
    display(n2-n1)

test1 = test[~np.isnan(test["sequenceNo_x"])]
test1 = test1[~np.isnan(test1["sequenceNo_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

2229
2210
7895627
7895627
7895627
-----------------------------------------------
1077.0
3244.0
10102.0
340.5
768.0
1662.25
0.9999873347613812
1.2665238618794935e-05


In [1]:
# Order + trade
import pandas as pd
import random
import numpy as np
import pickle

TradeLog = pd.read_csv(r'F:\data\20200730\postMDGW_OES\mdTradeLog_20200730_0913.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "mdSource", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
TradeLog = TradeLog[TradeLog['mdSource'] == 4]
OrderLog = pd.read_csv(r'F:\data\20200730\postMDGW_OES\mdOrderLog_20200730_0913.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "mdSource", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
OrderLog = OrderLog[OrderLog['mdSource'] == 4]
TradeLog1 = pd.read_csv(r'F:\data\20200730\preMDGW\EXE\Logs_full\mdTradeLog_20200730_0929.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "mdSource", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
OrderLog1 = pd.read_csv(r'F:\data\20200730\preMDGW\EXE\Logs_full\mdOrderLog_20200730_0929.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "mdSource", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]

stocks = OrderLog1["SecurityID"].unique()
l1 = stocks[:1000]
l2 = stocks[1000:]
OrderLogSZ = OrderLog[OrderLog["SecurityID"].isin(l1)]
OrderLogSZ1 = OrderLog1[OrderLog1["SecurityID"].isin(l1)]
print(len(OrderLogSZ["SecurityID"].unique()))
print(len(OrderLogSZ1["SecurityID"].unique()))
OrderLogSZZ = OrderLog[OrderLog["SecurityID"].isin(l2)]
OrderLogSZZ1 = OrderLog1[OrderLog1["SecurityID"].isin(l2)]
print(len(OrderLogSZZ["SecurityID"].unique()))
print(len(OrderLogSZZ1["SecurityID"].unique()))
del OrderLog
del OrderLog1
TradeLogSZ = TradeLog[TradeLog["SecurityID"].isin(l1)]
TradeLogSZ1 = TradeLog1[TradeLog1["SecurityID"].isin(l1)]
print(len(TradeLogSZ["SecurityID"].unique()))
print(len(TradeLogSZ1["SecurityID"].unique()))
TradeLogSZZ = TradeLog[TradeLog["SecurityID"].isin(l2)]
TradeLogSZZ1 = TradeLog1[TradeLog1["SecurityID"].isin(l2)]
print(len(TradeLogSZZ["SecurityID"].unique()))
print(len(TradeLogSZZ1["SecurityID"].unique()))
del TradeLog
del TradeLog1

OrderLogSZ.to_pickle(r'F:\\data\\20200730\\baseline\\order1.pkl')
OrderLogSZZ.to_pickle(r'F:\\data\\20200730\\baseline\\order2.pkl')
TradeLogSZ.to_pickle(r'F:\\data\\20200730\\baseline\\trade1.pkl')
TradeLogSZZ.to_pickle(r'F:\\data\\20200730\\baseline\\trade2.pkl')
OrderLogSZ1.to_pickle(r'F:\\data\\20200730\\test\\order1.pkl')
OrderLogSZZ1.to_pickle(r'F:\\data\\20200730\\test\\order2.pkl')
TradeLogSZ1.to_pickle(r'F:\\data\\20200730\\test\\trade1.pkl')
TradeLogSZZ1.to_pickle(r'F:\\data\\20200730\\test\\trade2.pkl')

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1000
1000
1210
1210
1000
1000
1210
1210


In [1]:
import pandas as pd
import random
import numpy as np
import pickle

F1 = open("F:\\data\\20200730\\baseline\\trade1.pkl", 'rb')
TradeLogSZ = pickle.load(F1)
F2 = open("F:\\data\\20200730\\test\\trade1.pkl", 'rb')
TradeLogSZ1 = pickle.load(F2)

sl = list(set(TradeLogSZ["SecurityID"].unique()) & set(TradeLogSZ['SecurityID'].unique()))
TradeLogSZ = TradeLogSZ[TradeLogSZ["SecurityID"].isin(sl)]
TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1["SecurityID"].isin(sl)]
print(len(TradeLogSZ["SecurityID"].unique()))
print(len(TradeLogSZ1["SecurityID"].unique()))

F1 = open("F:\\data\\20200730\\baseline\\trade2.pkl", 'rb')
t = pickle.load(F1)
F2 = open("F:\\data\\20200730\\test\\trade2.pkl", 'rb')
t1 = pickle.load(F2)
sl = list(set(t["SecurityID"].unique()) & set(t1['SecurityID'].unique()))
t = t[t["SecurityID"].isin(sl)]
t1 = t1[t1["SecurityID"].isin(sl)]
print(len(t["SecurityID"].unique()))
print(len(t1["SecurityID"].unique()))

TradeLogSZ = pd.concat([TradeLogSZ, t])
TradeLogSZ1 = pd.concat([TradeLogSZ1, t1])
del t
del t1

TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

TradeLogSZ = TradeLogSZ[TradeLogSZ['TransactTime'] >= 93000000]
TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1['TransactTime'] >= 93000000]

columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

re = re[~re["sequenceNo_x"].isnull()]
re = re[~re["sequenceNo_y"].isnull()]
diff = re["clockAtArrival_y"] - re["clockAtArrival_x"]
re["diff"] = re["clockAtArrival_y"] - re["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))
print(sum(diff == 0)/len(diff))

1000
1000
1210
1210
45533443
45533419
45533443
-----------------------------------------------


'test is not complete:'

,clockAtArrival_x,sequenceNo_x,exchId_x,mdSource_x,TransactTime,ApplSeqNum,SecurityID,ExecType,TradeBSFlag,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum,clockAtArrival_y,sequenceNo_y,exchId_y,mdSource_y
30900152,1596092409472872,251435772,2,4,150000000,25001759,300836,F,N,635400,1900,1207260000,24929326,24922275,NaN,NaN,NaN,NaN
30900153,1596092409472874,251435774,2,4,150000000,25001760,300836,F,N,635400,600,381240000,24836134,24922275,NaN,NaN,NaN,NaN
30900154,1596092409472877,251435776,2,4,150000000,25001761,300836,F,N,635400,200,127080000,24843925,24922275,NaN,NaN,NaN,NaN
30900155,1596092409472878,251435777,2,4,150000000,25001762,300836,F,N,635400,100,63540000,24855802,24922275,NaN,NaN,NaN,NaN
30900156,1596092409472882,251435781,2,4,150000000,25001763,300836,F,N,635400,200,127080000,24860194,24922275,NaN,NaN,NaN,NaN
30900157,1596092409472885,251435783,2,4,150000000,25001764,300836,F,N,635400,100,63540000,24861108,24922275,NaN,NaN,NaN,NaN
30900158,1596092409472890,251435784,2,4,150000000,25001765,300836,F,N,635400,200,127080000,24866997,24922275,NaN,NaN,NaN,NaN
30900159,1596092409472891,251435785,2,4,150000000,25001766,300836,F,N,635400,800,508320000,24870040,24922275,NaN,NaN,NaN,NaN
30900160,1596092409472892,251435786,2,4,150000000,25001767,300836,F,N,635400,200,127080000,24874955,24922275,NaN,NaN,NaN,NaN
30900161,1596092409472894,251435787,2,4,150000000,25001768,300836,F,N,635400,200,127080000,24880082,24922275,NaN,NaN,NaN,NaN


24
[150000000]
1
[300836]
730.0
858.0
1560.0
818.75
1067.0
1238.0
0.9999894363302698
1.054170783880736e-05
2.1961891330848667e-08


In [1]:
import pandas as pd
import random
import numpy as np
import pickle

F1 = open("F:\\data\\20200730\\baseline\\order1.pkl", 'rb')
OrderLogSZ = pickle.load(F1)
F2 = open("F:\\data\\20200730\\test\\order1.pkl", 'rb')
OrderLogSZ1 = pickle.load(F2)

OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
OrderLogSZ1["OrderType"] = OrderLogSZ1["OrderType"].apply(lambda x: str(x))
sl = list(set(OrderLogSZ["SecurityID"].unique()) & set(OrderLogSZ1['SecurityID'].unique()))
OrderLogSZ = OrderLogSZ[OrderLogSZ["SecurityID"].isin(sl)]
OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["SecurityID"].isin(sl)]
print(len(OrderLogSZ["SecurityID"].unique()))
print(len(OrderLogSZ1["SecurityID"].unique()))

F1 = open("F:\\data\\20200730\\baseline\\order2.pkl", 'rb')
o = pickle.load(F1)
F2 = open("F:\\data\\20200730\\test\\order2.pkl", 'rb')
o1 = pickle.load(F2)
o["OrderType"] = o["OrderType"].apply(lambda x: str(x))
o1["OrderType"] = o1["OrderType"].apply(lambda x: str(x))
sl = list(set(o["SecurityID"].unique()) & set(o1['SecurityID'].unique()))
o = o[o["SecurityID"].isin(sl)]
o1 = o1[o1["SecurityID"].isin(sl)]
print(len(o["SecurityID"].unique()))
print(len(o1["SecurityID"].unique()))

OrderLogSZ = pd.concat([OrderLogSZ, o])
OrderLogSZ1 = pd.concat([OrderLogSZ1, o1])
del o
del o1

OrderLogSZ = OrderLogSZ[OrderLogSZ['TransactTime'] >= 93000000]
OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1['TransactTime'] >= 93000000]

columns = ["TransactTime", "ApplSeqNum","Side",'OrderType', 'Price', 'OrderQty', "SecurityID"]
ree = pd.merge(OrderLogSZ, OrderLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = ree["sequenceNo_x"].count()
n2 = ree["sequenceNo_y"].count()
len1 = len(ree)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(ree[np.isnan(ree["sequenceNo_y"])])
    print(len(ree[np.isnan(ree["sequenceNo_y"])]))
    print(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
    print(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("test is complete, baseline is not complete:")
    display(ree[np.isnan(ree["sequenceNo_x"])])
    print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
    print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)
    
ree = ree[~ree["sequenceNo_x"].isnull()]
ree = ree[~ree["sequenceNo_y"].isnull()]
ree["diff"] = ree["clockAtArrival_y"] - ree["clockAtArrival_x"]
diff = ree["clockAtArrival_y"] - ree["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

1000
1000
1210
1210
49861685
49861709
49861709
-----------------------------------------------


'test is complete, baseline is not complete:'

,clockAtArrival_x,sequenceNo_x,exchId_x,mdSource_x,TransactTime,ApplSeqNum,SecurityID,Side,OrderType,Price,OrderQty,clockAtArrival_y,sequenceNo_y,exchId_y,mdSource_y
49861685,NaN,NaN,NaN,NaN,145959960,24931711,166,1,2,57700,200,1596092402868460,103681653,2,24
49861686,NaN,NaN,NaN,NaN,145959970,24931712,2885,1,2,171800,1200,1596092402870869,103681656,2,24
49861687,NaN,NaN,NaN,NaN,145959970,23810406,2465,1,2,146000,1000,1596092402871472,103681657,2,24
49861688,NaN,NaN,NaN,NaN,145959970,23810407,2923,1,2,231600,200,1596092402876179,103681658,2,24
49861689,NaN,NaN,NaN,NaN,145959970,23810408,2567,1,2,122500,5000,1596092402876908,103681659,2,24
49861690,NaN,NaN,NaN,NaN,145959970,24872064,2506,1,2,30900,600,1596092402877115,103681660,2,24
49861691,NaN,NaN,NaN,NaN,145959970,24236736,300664,2,2,102600,200,1596092402878335,103681661,2,24
49861692,NaN,NaN,NaN,NaN,145959970,23810409,300370,1,2,25600,3000,1596092402878751,103681662,2,24
49861693,NaN,NaN,NaN,NaN,145959970,24931713,300059,2,2,261000,1000,1596092402880325,103681663,2,24
49861694,NaN,NaN,NaN,NaN,145959970,24236737,961,2,2,95500,3100,1596092402880871,103681664,2,24


[145959960 145959970 145959980]
24
[   166   2885   2465   2923   2567   2506 300664 300370 300059    961
    625    540   2626   2428   2286 300775   2069   2602 300045 300759
   2457 300589   2208   2299]


24

708.0
828.0
1569.0
431.5
1002.0
1229.5
0.9999883277109468
1.1531900696897828e-05
